In [1]:
from DatasetCreation import DatasetCreation
import numpy as np

In [2]:
dsc = DatasetCreation('/media/philipp/DATA/2018_tamsweg/')

In [3]:
# set input data directory containing the preprocessed data
dsc.set_input_dir('/media/philipp/DATA/2018_bischofshofen/')
# check for correctness
dsc.path_dir

'/media/philipp/DATA/2018_bischofshofen/'

In [4]:
data_dic = {'ortho':{'dtype':np.uint8, 'dim':4}, \
              'dsm':{'dtype':np.float16, 'dim':1}, \
              'dtm':{'dtype':np.float16, 'dim':1}, \
              'slope':{'dtype':np.float16, 'dim':1}, \
              'ground_truth':{'dtype':np.uint8, 'dim':1}}

# set data types to be included
dsc.set_data_types(data_dic)
# check for correctness
dsc.data_dic

{'ortho': {'dtype': numpy.uint8, 'dim': 4},
 'dsm': {'dtype': numpy.float16, 'dim': 1},
 'dtm': {'dtype': numpy.float16, 'dim': 1},
 'slope': {'dtype': numpy.float16, 'dim': 1},
 'ground_truth': {'dtype': numpy.uint8, 'dim': 1}}

In [5]:
## create new file
# set output path
#path_file = "/media/philipp/DATA/dataset/dataset_256_df_1.h5"
# create hdf5 file
#dsc.create_hdf5(path_file, tile_size=256, dataset_size=75000)

# set output path
path_hdf5 = '/media/philipp/DATA/dataset/dataset_256_df_1.h5'
# set hdf5 file
dsc.set_hdf5(path_hdf5)
# check for correctness
dsc.path_hdf5

'/media/philipp/DATA/dataset/dataset_256_df_1.h5'

In [6]:
dsc.add_dataset_to_hdf5(block_size=1000, start_index=39280)

0 1000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
43157
1000 2000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
47017
2000 3000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
50869
3000 4000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
54722
4000 5000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
58593
5000 6000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
62452
6000 7000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
66312
7000 8000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
70176
8000 9000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000

NameError: name 'dsc' is not defined

In [ ]:
# tamsweg (177)         =     0:39280
# bischofshofen (176)   = 39280:74552

In [ ]:
# calc stats

In [1]:
from DatasetCreation import DatasetStats
import numpy as np
import h5py

In [2]:
stats = DatasetStats('/media/philipp/DATA/2018_tamsweg/')

In [3]:
path_hdf5 = '/media/philipp/DATA/dataset/dataset_256_df_1.h5'
hdf5_ds = h5py.File(path_hdf5, 'r')

In [5]:
data_type='dtm'
start = 0
end = 74552

mean = stats.calc_mean(dset=hdf5_ds, data_type=data_type, start_index=start, end_index=end, block_size=10000, bands=1)
print(mean)
std = stats.calc_std(mean, dset=hdf5_ds, data_type=data_type, start=start, end=end, block_size=10000, bands=1, scaler=1000)
print(std)

[1281.435025217298]
[314.3229959492675]


In [6]:
hdf5_ds['dsm']

<HDF5 dataset "dsm": shape (75000, 256, 256, 1), type "<f2">

In [16]:
path_hdf5 = '/home/philipp/Data/edin_dataset/dataset_test_2.h5'
with h5py.File(path_hdf5, 'a') as hdf5_ds:
    hdf5_ds['x_ortho'].attrs['mean'] = np.array([13.807993078656509])
    hdf5_ds['x_ortho'].attrs['std'] = [10.449318123686943]

In [17]:
path_hdf5 = '/home/philipp/Data/edin_dataset/dataset_test_2.h5'
hdf5_ds = h5py.File(path_hdf5, 'r')

In [18]:
hdf5_ds.keys()

<KeysViewHDF5 ['x_ortho']>

In [19]:
hdf5_ds['x_ortho'].attrs.keys()

<KeysViewHDF5 ['mean', 'std']>

In [20]:
hdf5_ds['x_ortho'].attrs['mean']

array([13.80799308])

In [21]:
hdf5_ds['x_ortho'].attrs['std']

array([10.44931812])

In [22]:
hdf5_ds.close()

In [ ]:
mean_dsm = [13.807993078656509]
std_dsm = [10.449318123686943]

mean_dtm = []
std_dtm = []

mean_slope = [21.550225010730767]
std_slope = [11.841692292433436]

In [ ]:
### end ###

In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr
from osgeo import gdalconst

from glob import glob
import logging
import os

from DatasetCreation import DatasetCreation

In [2]:
data_dic = {'ortho':{'dtype':np.uint8, 'dim':4}, \
              'dsm':{'dtype':np.float16, 'dim':1}, \
              'dtm':{'dtype':np.float16, 'dim':1}, \
              'slope':{'dtype':np.float16, 'dim':1}, \
              'ground_truth':{'dtype':np.uint8, 'dim':1}}

data_types = ['ortho', 'dsm', 'dtm', 'slope', 'ground_truth']

tile_size = 256

dataset_size = 40000

block_size = 1000

In [3]:
dsc = DatasetCreation('/media/philipp/DATA/2018_tamsweg/')

In [9]:
paths = dsc.find_files('/media/philipp/DATA/2018_bischofshofen/', ['ortho', 'dsm', 'dtm', 'slope', 'ground_truth'])

In [6]:
"""
path_hdf5 = '/media/philipp/DATA/dataset/dataset_df_0.h5'

hdf5_ds = h5py.File(path_hdf5, 'a')

for data_type in data_types:
    x = hdf5_ds.create_dataset(data_type, \
    (dataset_size, tile_size, tile_size, data_dic[data_type]['dim']), \
    dtype=data_dic[data_type]['dtype'])
hdf5_ds.close()
"""

"\npath_hdf5 = '/media/philipp/DATA/dataset/dataset_df_0.h5'\n\nhdf5_ds = h5py.File(path_hdf5, 'a')\n\nfor data_type in data_types:\n    x = hdf5_ds.create_dataset(data_type,     (dataset_size, tile_size, tile_size, data_dic[data_type]['dim']),     dtype=data_dic[data_type]['dtype'])\nhdf5_ds.close()\n"

In [4]:
path_hdf5 = '/media/philipp/DATA/dataset/dataset_256_df_1.h5'
hdf5_ds = h5py.File(path_hdf5, 'a')

In [5]:
hdf5_ds['dsm']

<HDF5 dataset "dsm": shape (75000, 256, 256, 1), type "<f2">

In [12]:
ds_size = len(paths[list(paths.keys())[0]])
print(ds_size)

9135


In [11]:
if ds_size > block_size:
    counter = 0
    for i in range(ds_size // block_size):

        print(block_size*i, block_size*(i+1))

        data = dsc.prepare_dataset(paths, block_size*i, block_size*(i+1), data_dic)

        for data_type in data_types:
            # set dataset
            x = hdf5_ds[data_type]
            # assign data
            x[counter:counter+data[data_type].shape[0],:,:,:] = data[data_type]
            
        counter += data[data_type].shape[0]
        print(counter)

    print(block_size*(i+1), block_size*(i+1) + ds_size % block_size)

0 1000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
3662
1000 2000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
7345
2000 3000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
11043
3000 4000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
14701
4000 5000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
18353
5000 6000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
22033
6000 7000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
25702
7000 8000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
29366
8000 9000
(1000, 512, 512, 4)
(1000, 512, 512, 1)
(1000, 512, 512, 1)
(1000, 

In [13]:
data = dsc.prepare_dataset(paths, 9000, 9135, data_dic)
counter = 74038

for data_type in dsc.data_types:
    # set dataset
    x = hdf5_ds[data_type]
    # assign data
    x[counter:counter+data[data_type].shape[0],:,:,:] = data[data_type]

counter += data[data_type].shape[0]
print(counter)

(135, 512, 512, 4)
(135, 512, 512, 1)
(135, 512, 512, 1)
(135, 512, 512, 1)
(135, 512, 512, 1)
74552


In [18]:
hdf5_ds['ortho'][74552]

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

In [6]:
hdf5_ds.close()